In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import pathlib
from tensorflow.keras import layers
from google.colab import drive
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
#Parameters
img_height = 200
img_width = 200

num_classes = 29
num_samples = 2048
batch_size = 32

In [3]:
# Function to create model, required for KerasClassifier
def create_model(learn_rate=0.01, neurons = 128):
	# create model
	model = tf.keras.Sequential([
		layers.experimental.preprocessing.Rescaling(1./255),
		layers.Conv2D(32, 3, activation='relu'),
		layers.MaxPooling2D(),
		layers.Conv2D(32, 3, activation='relu'),
		layers.MaxPooling2D(),
		layers.Conv2D(32, 3, activation='relu'),
		layers.MaxPooling2D(),
		layers.Flatten(),
		layers.Dense(neurons, activation='relu'),
		layers.Dense(num_classes)
	])
 
	# Compile model
	adam = tf.keras.optimizers.Adam(learning_rate=learn_rate)
	model.compile(
		optimizer = adam,
		loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True),
		metrics=['accuracy'])
	
	return model

In [4]:
#Importing the dataset
data_dir = "/content/gdrive/My Drive/Capstone/Dataset/test_set"
data_dir = pathlib.Path(data_dir)

ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  image_size=(img_height, img_width))

Found 8702 files belonging to 29 classes.


In [5]:
images = np.zeros(shape=(num_samples, img_height, img_width, 3))
labels = np.zeros(shape=(num_samples))

i = 0
for images_batch, labels_batch in ds.as_numpy_iterator():
  images[i*batch_size : (i+1)*batch_size] = images_batch
  labels[i*batch_size : (i+1)*batch_size] = labels_batch
  print("Loading batch {} done".format(i))
  if (i + 1) * batch_size == num_samples:
    print("Finished loading samples for grid search")
    break
  i += 1

Loading batch 0 done
Loading batch 1 done
Loading batch 2 done
Loading batch 3 done
Loading batch 4 done
Loading batch 5 done
Loading batch 6 done
Loading batch 7 done
Loading batch 8 done
Loading batch 9 done
Loading batch 10 done
Loading batch 11 done
Loading batch 12 done
Loading batch 13 done
Loading batch 14 done
Loading batch 15 done
Loading batch 16 done
Loading batch 17 done
Loading batch 18 done
Loading batch 19 done
Loading batch 20 done
Loading batch 21 done
Loading batch 22 done
Loading batch 23 done
Loading batch 24 done
Loading batch 25 done
Loading batch 26 done
Loading batch 27 done
Loading batch 28 done
Loading batch 29 done
Loading batch 30 done
Loading batch 31 done
Loading batch 32 done
Loading batch 33 done
Loading batch 34 done
Loading batch 35 done
Loading batch 36 done
Loading batch 37 done
Loading batch 38 done
Loading batch 39 done
Loading batch 40 done
Loading batch 41 done
Loading batch 42 done
Loading batch 43 done
Loading batch 44 done
Loading batch 45 don

In [9]:
# create model
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=1)

# define the grid search parameters
# learn_rate = [0.01, 0.1, 0.2, 0.3]
# batch_sizes = [20, 40, 60, 80, 100]
# epochs = [1, 5, 10]
learn_rate = [0.01, 0.1, 0.2]
batch_sizes = [16, 32, 64]
epochs = [1, 5, 10]
neurons = [64, 128, 256]
param_grid = dict(batch_size=batch_sizes, epochs=epochs, learn_rate=learn_rate, neurons=neurons)

# grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-2, cv=3)
grid_result = grid.fit(images, labels)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


32/32 [==============================] - 1s 22ms/step - loss: 5.4665 - accuracy: 0.0332
Best: 0.051770 using {'batch_size': 64, 'epochs': 1, 'learn_rate': 0.01, 'neurons': 128}
0.033201 (0.006576) with: {'batch_size': 16, 'epochs': 1, 'learn_rate': 0.01, 'neurons': 64}
0.034177 (0.004182) with: {'batch_size': 16, 'epochs': 1, 'learn_rate': 0.01, 'neurons': 128}
0.035157 (0.001217) with: {'batch_size': 16, 'epochs': 1, 'learn_rate': 0.01, 'neurons': 256}
0.040037 (0.002464) with: {'batch_size': 16, 'epochs': 1, 'learn_rate': 0.1, 'neurons': 64}
0.035157 (0.004316) with: {'batch_size': 16, 'epochs': 1, 'learn_rate': 0.1, 'neurons': 128}
0.028809 (0.008973) with: {'batch_size': 16, 'epochs': 1, 'learn_rate': 0.1, 'neurons': 256}
0.035650 (0.008512) with: {'batch_size': 16, 'epochs': 1, 'learn_rate': 0.2, 'neurons': 64}
0.034675 (0.009770) with: {'batch_size': 16, 'epochs': 1, 'learn_rate': 0.2, 'neurons': 128}
0.031741 (0.003676) with: {'batch_size': 16, 'epochs': 1, 'learn_rate': 0.2, 'n